# Práctica Independiente

En esta práctica vamos a utilizar el modelo que guardamos en la carpeta model en la práctica anterior y vamos a implementarlo en el Tensorflow Serving Container.

Paso 1. Importemos las librerías

In [1]:
import os
import sagemaker
from sagemaker.tensorflow.serving import Model
import tensorflow as tf
import numpy as np
from sagemaker import get_execution_role
import numpy as np
import cv2
from matplotlib import pyplot as mlt
import boto3
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Paso 2. SageMaker requiere que construyamos un archivo tar.gz para pasárselo al tensorflow serving container. Recuerden que el container puede recibir varios modelos, por eso al guardar el modelo lo pusimos en una carpeta llamada /1 y ahora vamos a comprimirla en formato .tar.gz.

In [2]:
%%bash
tar -czvf model.tar.gz 1

1/
1/assets/
1/saved_model.pb
1/variables/
1/variables/variables.index
1/variables/variables.data-00000-of-00001


Paso 3. Tenemos que subir el model.tar.gz a S3 y guardarnos la ubicación en un string. Pueden usar el SDK a través de sagemaker.Session().upload_data o investigar el comando bash aws s3 cp

In [3]:
s3_prefix = 'dh-serving/model'

model_data = sagemaker.Session().upload_data(path='model.tar.gz', key_prefix=s3_prefix)

In [4]:
model_data

's3://sagemaker-us-east-1-452432741922/dh-serving/model/model.tar.gz'

Paso 4. Utilizar el SDK para crear el tensorflow_serving_model

In [5]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket = sagemaker_session.default_bucket()

In [6]:
# The "Model" object doesn't create a SageMaker Model until a Transform Job or Endpoint is created.
tensorflow_serving_model = Model(model_data=model_data,
                                 role=role,
                                 framework_version='2.0',
                                 sagemaker_session=sagemaker_session)

Paso 4. Utilizar el método deploy para desplegar el modelo sobre una infraestructura. Como tipo de instancia pueden elegir el ml.m5.large

In [7]:
predictor = tensorflow_serving_model.deploy(initial_instance_count=1,
                                            instance_type='ml.m5.large')

-------------!

Paso 5. Carguemos en memoria los features y labels de test en variables utilizando np.load

In [8]:
test_features = np.load('test_features.npy')
test_labels = np.load('test_labels.npy')

In [9]:
test_features.shape

(1000, 8192)

Paso 6: Crear el payload para el modelo. Para eso necesario armar un diccionario con la key instances y una lista de instancias. Los numpy arrays no son serializables. Para hacerlos serializables, es necesario utilizar el método tolist().

Recurden armar un predict_batch con una de las instancias. El shape de la variable predict_batch debería ser (1, 8192)

In [10]:
predict_batch = test_features[0:1,:]

In [11]:
payload = {
          'instances': predict_batch.tolist()
           }

Paso 7: Pueden utilizar el método predict() del predictor para obtener las predicciones

In [12]:
resp = predictor.predict(payload)['predictions']

In [13]:
np.array(resp)

array([[0.99954152]])